In [ ]:
# imports
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import cv2
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow import data as tf_data
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, log_loss
import numpy as np
from typing import List, Dict, Any, Union

print("Modules imported")

In [ ]:
# variables
all_labels = ['nature', 'country', 'city']
path = "../datasets/all_data/entropy_results_short.json"
parallel_jobs = 5

# hyperparameters
test_part = 0.05
epochs = 2
batch_size = 64
learning_rate = 0.01

# check gpu
devices = tf.config.list_physical_devices()
print("Available devices:")
for device in devices:
    print(device.name)
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"The model will run on GPU: {physical_devices[0].name}")
else:
    print("No GPU found, the model will run on CPU.")